# Meal Plan Project 

Currently, meal plans are writeen to a word document weekly  
let's automate this process by:  
1.*Scraping meal plan from website*  
2.*Writing it to a word document and formatting the text*  
**BONUS:** *Email and/or print automatically*

## Task: Pull meal plan from site and write to word document

The app I use has no API, so I followed [this tutorial](https://www.youtube.com/watch?v=XQgXKtPSzUI) to find the process and tools used to scrape contents from a site

### Import libraries

In [1]:
#Library for fetching a webpage
from urllib.request import urlopen as request

In [2]:
#Library for parsing html from webpage
from bs4 import BeautifulSoup as soup 

In [3]:
#Library for writing to a word document
import docx
from docx.shared import Pt #for adjusting font and size
from docx.enum.text import WD_ALIGN_PARAGRAPH #for changing paragraph alignment
from docx.shared import Inches, Cm #for changing page margains 

In [4]:
#Library to pull HTML content from Java enabled sites
from selenium import webdriver

In [5]:
#Needed to program in waiting for elements to appear before moving on with script
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [6]:
import re # For regular expressions. Will use to get date of meal plan

In [7]:
import os #for saving docs 

In [8]:
import smtplib #for sending emails
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

In [9]:
import time #add wait time to code 

***Create a input validation function I will use later***

In [10]:
def val_loop(data):
    count = 0
    while count != 20:
        if data == None or len(data) == 0:
            count += 1
    if count == 20:
        print('Parsing error')
        
# I made this function and ended up taking it back out.
# I got stuck in an infinite loop I think at the beginning of the main block of code
# And it seems the root cause of the issue was that I wasn't giving all the other pages I was navigating to enough time to load
# Just like with my main page divs
# So, I added a 5 second wait after loading any page before saving the source and that has seemed to work.

### I first attempted to scrape the site using BeautifulSoup

In [11]:
#Saving our URL as a variable
my_url = 'https://www.strongrfastr.com/app/meal_plan/next'

#Downloading the webpage
uClient = request(my_url) 

#Saving html contents of webpage to a variable
page_html = uClient.read()

#Closing connection
uClient.close() 

#Parsing webpage HTML and saving as a soup object
page_soup = soup(page_html,"html5lib")  

page_soup.body

<body><noscript>You need to enable JavaScript to run this app.</noscript><div id="root"></div><script src="https://js.stripe.com/v3/"></script><script>!function(e){function t(t){for(var n,a,i=t[0],c=t[1],f=t[2],s=0,p=[];s<i.length;s++)a=i[s],Object.prototype.hasOwnProperty.call(o,a)&&o[a]&&p.push(o[a][0]),o[a]=0;for(n in c)Object.prototype.hasOwnProperty.call(c,n)&&(e[n]=c[n]);for(l&&l(t);p.length;)p.shift()();return u.push.apply(u,f||[]),r()}function r(){for(var e,t=0;t<u.length;t++){for(var r=u[t],n=!0,i=1;i<r.length;i++){var c=r[i];0!==o[c]&&(n=!1)}n&&(u.splice(t--,1),e=a(a.s=r[0]))}return e}var n={},o={1:0},u=[];function a(t){if(n[t])return n[t].exports;var r=n[t]={i:t,l:!1,exports:{}};return e[t].call(r.exports,r,r.exports,a),r.l=!0,r.exports}a.e=function(e){var t=[],r=o[e];if(0!==r)if(r)t.push(r[2]);else{var n=new Promise((function(t,n){r=o[e]=[t,n]}));t.push(r[2]=n);var u,i=document.createElement("script");i.charset="utf-8",i.timeout=120,a.nc&&i.setAttribute("nonce",a.nc),i.src=

**I got a message saying I needed to enable javascript. After some research, I decided to attempt this using Selenium as well**

## Scrape using Selenium webdriver and BeautifulSoup

**Because webdriver does not use my account with saved credentials, I will need to navigate to the site, then log into the app**

In [12]:
url = 'https://www.strongrfastr.com/app/meal_plan/next' #save the url we want to open to variable 'url'

In [13]:
#Open a browser window
driver = webdriver.Firefox()

In [14]:
#Navigate to meal planning site
driver.get(url) 

In [15]:
#Wait for the email login button to load
try:
    element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, 'email-login-btn'))
    )
except:
    driver.quit() 

In [16]:
#Define my email and password
User_Name = 'SomeUserName'
PWD = 'SomePassword'

In [17]:
#Define login button, click button, pass in login credentials
Login_button = driver.find_element_by_id('email-login-btn')
Login_button.click()

username = driver.find_element_by_id("email-field")
password = driver.find_element_by_id("password-field")

username.send_keys(User_Name)
password.send_keys(PWD) 

In [18]:
#Wait for sign in button to load
try:
    element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, 'submit-email-btn'))
    )
except:
    driver.quit() 

In [19]:
#Define sign in button, click to proceed to my profile and next week's meal plan
submit_button = driver.find_element_by_id('submit-email-btn')
submit_button.click() 

### Next, scrape this page

In [20]:
#Wait for the root div to load (the element that contains the rest of the webpage)
try:
    element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, 'root'))
    )
except:
    driver.quit() 

In [21]:
#Use time to give page a couple seconds to load
time.sleep(5) # Sleep for 5 seconds
#Save page to variable
source = driver.page_source

In [22]:
#Use BeautifulSoup to parse the webpage
page_soup = soup(source, "html.parser")

In [23]:
#print the webpage HTML
page_soup.body

<body><noscript>You need to enable JavaScript to run this app.</noscript><div id="root"><div style="position: relative;"><div><div class="page-container off-white" style="min-height: 100vh;"><div class="app-scroll-container" id="app-scroll-container-uzsa3i"><div class="nav-container activity-tabs-container-extra"><div><div class="limit-50-h display-flex"><div class="hide-on-med-and-down nutrition-background" style="min-width: 360px; padding-left: 10px; padding-right: 10px; width: 25%;"><div style="position: fixed; min-width: 360px; padding-left: 10px; padding-right: 10px; width: 25%; left: 0px;"><div class="valign-wrapper mt20 nutrition-header-wrapper"><div class="meal-plan-heading-2 mr10">Nutrition</div><a class="waves-effect btn btn-rounded btn-primary btn-outlined btn-no-shadow" href="/app/settings/nutrition"><div class="btn-content"><svg aria-hidden="true" class="svg-inline--fa fa-edit fa-w-18" data-icon="edit" data-prefix="far" focusable="false" role="img" viewbox="0 0 576 512" xm

### Now, I can try to extract the meal data using the HTML elements of this page

**The first webpage contains an overview with meals and dish names. Each meal has it's own webpage, which in turn has a page for each dish. I need the Meal>Dish>Recipe for all 4 meals and all dishes to be written to a word document.**

The order of operations for this code should be as follows:
1. Get a list with all 4 daily meals in it
2. For each meal in the list (Breakfast, Lunch, Dinner, Snack) print the type of meal, and its dishes
3. For each meal in the list, click on it to visit its webpage
4. Get a list of all dishes on page
5. For each dish, click on its page, scrape dish name and ingredients and write to word

I will write the whole thing out with print statements for testing, then start adding in docx code to write to word

In [24]:
# Test Block: Will print error of I scraped a not yet loaded page
empty_list = [0,1,2,3,4,5]
newlist = []
if len(empty_list) > 0:
    newlist = empty_list[:4]
else:
    print('ERROR:Page not loaded before scraped')

newlist

[0, 1, 2, 3]

In [25]:
#Save all meal cards to variable, save the first 4 (One day's meals) to a variable

#object of all divs that contain a meal
mydivs = page_soup.find_all("div", {"class": "meal-cards-day-wrap"}) #BeautifulSoup's findall() method returns a list
meals = []

if len(mydivs) > 0:
    meals = mydivs[0:4] 
    #assign the first four meals (they're the same all week) to variable 'meals'. This is the list we will iterate over
else:
    print('ERROR:Page not loaded before scraped')
meals

[<div class="meal-cards-day-wrap"><div class="sideways-text">Breakfast</div><a class="card meal-cards-day-view buttonize-color-only" href="/app/weekly_meal/breakfast/2021-05-17/edit" id="daily-meal-11581084-card"><div class="position-relative"><div class="display-flex"><div class="image-meal-card-day position-relative"><img alt="" src="https://d2jbk7d41q2u2w.cloudfront.net/uploads/recipe/image/8160/medium_thumb_c45d88b242f7d62caf63906611990659-1595251380.jpg"/></div><div class="today-recipe-details" style="min-width: 0px;"><div class="meal-plan-recipe-title lh1">Desayuno Mexican</div><div style="font-size: 12px;"><div class="low-emphasis lh1 abbr-text">+ Coffee <br/></div></div></div></div></div></a><div class="meal-swap-btn button-placeholder buttonize no-shadow" id="meal-11581084-swap-btn"><svg aria-hidden="true" class="svg-inline--fa fa-exchange-alt fa-w-16" data-icon="exchange-alt" data-prefix="fas" focusable="false" role="img" viewbox="0 0 512 512" xmlns="http://www.w3.org/2000/sv

***First get the meal type and dish names for each meal. Print is stand in for write to docx***

In [26]:
for meal in meals: #for each meal in the meals list
    print(meal.div.text) #print the type of meal
    main_dishes = meal.find_all('div',{'class':'meal-plan-recipe-title'}) #create dictionary of all main dishes
    side_dishes = meal.find_all('div',{'class':'low-emphasis lh1 abbr-text'}) #create dictionary of all side dishes
    for dish in main_dishes:
        print(dish.text) #print all main dishes for this meal in the meals list
    for side in side_dishes:
        print(side.text) #print all side dishes for this meal in the meals list

Breakfast
Desayuno Mexican
+ Coffee 
Lunch
Chicken burrito
+ Roasted almonds+ Greek cucumber salad
Dinner
Buffalo chicken wings
+ Sweet potato fries
Snack
Grapes
+ Cottage cheese and pineapple


***Next, add steps to visit each meal page, then dish page and print those contents as well***

In [27]:
#for each meal, extract the end of it's URL, append the base URL and print it
links = []
for meal in meals:
    for link in meal.find_all('a'):
        links.append(link)
        print('https://www.strongrfastr.com' + link.get('href'))

https://www.strongrfastr.com/app/weekly_meal/breakfast/2021-05-17/edit
https://www.strongrfastr.com/app/weekly_meal/lunch/2021-05-17/edit
https://www.strongrfastr.com/app/weekly_meal/dinner/2021-05-17/edit
https://www.strongrfastr.com/app/weekly_meal/snack/2021-05-17/edit


In [28]:
# Get the date for the meal plan in question
link = links[0].get('href')

dateRegex = re.compile(r'\d\d\d\d-\d\d-\d\d') #defines pattern to search for, creates a regular expression object
date = dateRegex.search(link) #searchses for pattern and returns a match object to a variable
date_text = date.group() # the group() method returns a string we can save to the date_text variable
date_text

'2021-05-17'

In [29]:
#Now to try and visit each page and it's dish's pages
for meal in meals:
    for link in meal.find_all('a'):
        meal_url = 'https://www.strongrfastr.com' + link.get('href')
        driver.get(meal_url) #navigate to the URL for the meal on each iteration of the loop
        
        #Wait for the root div to load (the element that contains the rest of the webpage)
        try:
            element = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.ID, 'root'))
            )
        except:
            driver.quit()
        
        #Save page to variable
        meal_source = driver.page_source
        
        #Use BeautifulSoup to parse the webpage
        meal_page_soup = soup(meal_source, "html.parser")
        
        #BeautifulSoup's findall() method returns a list
        #object of all divs that contain a dish, and another of side dishes
        
        dish_divs = meal_page_soup.find_all("div", {"class": "mec-wrapper mb10"}) 
        side_dish_divs = meal_page_soup.find_all("div", {"class": "side-dish-container"})  
        
        
        #Print all main dishes
        for dish in dish_divs: # for every dish in the list of main dishes
            for link in dish.find_all('a'): #find all the links
                main_url = 'https://www.strongrfastr.com' + link.get('href') #assign url to variable
                driver.get(main_url) #navigate to the URL for the dish on each iteration of the loop
        
                #Wait for the root div to load (the element that contains the rest of the webpage)
                try:
                    element = WebDriverWait(driver, 20).until(
                        EC.presence_of_element_located((By.ID, 'root'))
                    )
                except:
                    driver.quit()
        
                #Save page to variable
                main_dish_source = driver.page_source
        
                #Use BeautifulSoup to parse the webpage
                main_dish_soup = soup(main_dish_source, "html.parser")
                
                
                #Scrape dish names
                dish_name_div = main_dish_soup.find("div", {"class": "max-list ml-center mt0"}) 
                dish_name = dish_name_div.find('h3').text
                
                #Scrape ingredients
                
                main_ingredients_list = main_dish_soup.find_all("div", {"class": "basic-heading"})
                
                #scrape quantities
                
                main_quantities_list = main_dish_soup.find_all("div", {"class": "basic-heading-sub"})
                
                print(dish_name)
                print()
                for i in range(len(main_ingredients_list)):
                    print(main_ingredients_list[i].text)
                    print(main_quantities_list[i].text)
                    print()
        
        #Repeat for side dishes
        for side_dish in side_dish_divs: # for every dish in the list of main dishes
            for side_link in side_dish.find_all('a'): #find all the links
                side_url = 'https://www.strongrfastr.com' + side_link.get('href') #assign url to variable
                driver.get(side_url) #navigate to the URL for the dish on each iteration of the loop
        
                #Wait for the root div to load (the element that contains the rest of the webpage)
                try:
                    element = WebDriverWait(driver, 20).until(
                        EC.presence_of_element_located((By.ID, 'root'))
                    )
                except:
                    driver.quit()
        
                #Save page to variable
                side_dish_source = driver.page_source
        
                #Use BeautifulSoup to parse the webpage
                side_dish_soup = soup(side_dish_source, "html.parser")
                
                
                #Scrape dish names
                side_name_div = side_dish_soup.find("div", {"class": "max-list ml-center mt0"}) 
                side_name = side_name_div.find('h3').text
                
                #Scrape ingredients
                
                side_ingredients_list = side_dish_soup.find_all("div", {"class": "basic-heading"})
                
                #scrape quantities
                
                side_quantities_list = side_dish_soup.find_all("div", {"class": "basic-heading-sub"})
                
                print(side_name)
                print()
                for s in range(len(side_ingredients_list)):
                    print(side_ingredients_list[s].text)
                    print(side_quantities_list[s].text)
                    print()

Desayuno Mexican

Coffee 

Almond Creamer (Silk)
205cals, 0p, 6f, 38c, 0fiber (per meal)

Organic Agave Nectar (Agave in The Raw)
3 oz (85g)

Coffee 
20 gram (20g)

Chicken burrito

Roasted almonds

Greek cucumber salad

Buffalo chicken wings

Sweet potato fries

Grapes

grapes
155cals, 2p, 1f, 25c, 10fiber (per meal)

Cottage cheese and pineapple

lowfat cottage cheese
186cals, 28p, 2f, 13c, 1fiber (per meal)

canned pineapple, drained
1 cup (226g)



***Now, put the last two steps together with print calls before proceeding to add code for writing to word***

In [30]:
# print meal titles and dishes

for meal in meals: #for each meal in the meals list
    print(meal.div.text) #print the type of meal
    main_dishes = meal.find_all('div',{'class':'meal-plan-recipe-title'}) #create dictionary of all main dishes
    side_dishes = meal.find_all('div',{'class':'low-emphasis lh1 abbr-text'}) #create dictionary of all side dishes
    for dish in main_dishes:
        print(dish.text) #print all main dishes for this meal in the meals list
    for side in side_dishes:
        print(side.text) #print all side dishes for this meal in the meals list
    print()
    
    
    #print ingredients and quantities
    
    for link in meal.find_all('a'):
        meal_url = 'https://www.strongrfastr.com' + link.get('href')
        driver.get(meal_url) #navigate to the URL for the meal on each iteration of the loop
        
        #Wait for the root div to load (the element that contains the rest of the webpage)
        try:
            element = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.ID, 'root'))
            )
        except:
            driver.quit()
        
        #Save page to variable
        meal_source = driver.page_source
        
        #Use BeautifulSoup to parse the webpage
        meal_page_soup = soup(meal_source, "html.parser")
        
        #BeautifulSoup's findall() method returns a list
        #object of all divs that contain a dish, and another of side dishes
        
        dish_divs = meal_page_soup.find_all("div", {"class": "mec-wrapper mb10"}) 
        side_dish_divs = meal_page_soup.find_all("div", {"class": "side-dish-container"})  
        
        
        #Print all main dishes
        for dish in dish_divs: # for every dish in the list of main dishes
            for link in dish.find_all('a'): #find all the links
                main_url = 'https://www.strongrfastr.com' + link.get('href') #assign url to variable
                driver.get(main_url) #navigate to the URL for the dish on each iteration of the loop
        
                #Wait for the root div to load (the element that contains the rest of the webpage)
                try:
                    element = WebDriverWait(driver, 20).until(
                        EC.presence_of_element_located((By.ID, 'root'))
                    )
                except:
                    driver.quit()
        
                #Save page to variable
                main_dish_source = driver.page_source
        
                #Use BeautifulSoup to parse the webpage
                main_dish_soup = soup(main_dish_source, "html.parser")
                
                
                #Scrape dish names
                dish_name_div = main_dish_soup.find("div", {"class": "max-list ml-center mt0"}) 
                dish_name = dish_name_div.find('h3').text
                
                #Scrape ingredients
                
                main_ingredients_list = main_dish_soup.find_all("div", {"class": "basic-heading"})
                
                #scrape quantities
                
                main_quantities_list = main_dish_soup.find_all("div", {"class": "basic-heading-sub"})
                
                print(dish_name)
                print()
                for i in range(len(main_ingredients_list)):
                    print(main_ingredients_list[i].text)
                    print(main_quantities_list[i].text)
                    print()
        
        #Repeat for side dishes
        for side_dish in side_dish_divs: # for every dish in the list of main dishes
            for side_link in side_dish.find_all('a'): #find all the links
                side_url = 'https://www.strongrfastr.com' + side_link.get('href') #assign url to variable
                driver.get(side_url) #navigate to the URL for the dish on each iteration of the loop
        
                #Wait for the root div to load (the element that contains the rest of the webpage)
                try:
                    element = WebDriverWait(driver, 20).until(
                        EC.presence_of_element_located((By.ID, 'root'))
                    )
                except:
                    driver.quit()
        
                #Save page to variable
                side_dish_source = driver.page_source
        
                #Use BeautifulSoup to parse the webpage
                side_dish_soup = soup(side_dish_source, "html.parser")
                
                
                #Scrape dish names
                side_name_div = side_dish_soup.find("div", {"class": "max-list ml-center mt0"}) 
                side_name = side_name_div.find('h3').text
                
                #Scrape ingredients
                
                side_ingredients_list = side_dish_soup.find_all("div", {"class": "basic-heading"})
                
                #scrape quantities
                
                side_quantities_list = side_dish_soup.find_all("div", {"class": "basic-heading-sub"})
                
                print(side_name)
                print()
                for s in range(len(side_ingredients_list)):
                    print(side_ingredients_list[s].text)
                    print(side_quantities_list[s].text)
                    print()
                    
                    
#Close the browser window
#driver.close()

Breakfast
Desayuno Mexican
+ Coffee 

Desayuno Mexican

eggs
529cals, 29p, 25f, 36c, 11fiber (per meal)

Refried Beans with Cheese 
2 1/4 large (113g)

Flour Tortilla (Mission)
189 3/4 g (190g)

Coffee 

Almond Creamer (Silk)
205cals, 0p, 6f, 38c, 0fiber (per meal)

Organic Agave Nectar (Agave in The Raw)
3 oz (85g)

Coffee 
20 gram (20g)

Lunch
Chicken burrito
+ Roasted almonds+ Greek cucumber salad

Chicken burrito

Roasted almonds

Greek cucumber salad

Dinner
Buffalo chicken wings
+ Sweet potato fries

Buffalo chicken wings

Sweet potato fries

Snack
Grapes
+ Cottage cheese and pineapple

Grapes

grapes
155cals, 2p, 1f, 25c, 10fiber (per meal)

Cottage cheese and pineapple

lowfat cottage cheese
186cals, 28p, 2f, 13c, 1fiber (per meal)

canned pineapple, drained
1 cup (226g)



***Now, I will write in steps for writing to word instead of printing***

In [31]:
#Change current working directory

os.chdir('C:\\Users\\Administrator.Sergio-PC\\Documents\\meal_plans')


#Create a word document 

doc = docx.Document()

# Set to narrow margins
sections = doc.sections
for section in sections:
    section.top_margin = Cm(2.54)
    section.bottom_margin = Cm(2.54)
    section.left_margin = Cm(2.54)
    section.right_margin = Cm(2.54)

date_p = doc.add_paragraph(date_text)

date_p.alignment = WD_ALIGN_PARAGRAPH.RIGHT

# Write meal titles and dishes

for meal in meals: #for each meal in the meals list
    title_p = doc.add_paragraph()
    
    title_p.add_run(meal.div.text + ': ')
    
    main_dishes = meal.find_all('div',{'class':'meal-plan-recipe-title'}) #create dictionary of all main dishes
    side_dishes = meal.find_all('div',{'class':'low-emphasis lh1 abbr-text'}) #create dictionary of all side dishes
    for dish in main_dishes:
        
        title_p.add_run(dish.text)
        
    for side in side_dishes:
        
        title_p.add_run(side.text)

    for run in title_p.runs:
        font = run.font

        font.name = 'Calibri'
        font.size = Pt(12)
        font.underline = True
    
    font = title_p.runs[0].font
    font.all_caps = True

    
    # Write all ingredients and quantities
    
    for link in meal.find_all('a'):
        meal_url = 'https://www.strongrfastr.com' + link.get('href')
        driver.get(meal_url) #navigate to the URL for the meal on each iteration of the loop
        
        #Wait for the root div to load (the element that contains the rest of the webpage)
        try:
            element = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.ID, 'root'))
            )
        except:
            driver.quit()
        
        #Use time to give page a couple seconds to load
        time.sleep(5) # Sleep for 5 seconds
        
        #Save page to variable
        meal_source = driver.page_source
        
        #Use BeautifulSoup to parse the webpage
        meal_page_soup = soup(meal_source, "html.parser")
        
        #BeautifulSoup's findall() method returns a list
        #object of all divs that contain a dish, and another of side dishes
        
        dish_divs = meal_page_soup.find_all("div", {"class": "mec-wrapper mb10"}) 
        side_dish_divs = meal_page_soup.find_all("div", {"class": "side-dish-container"})  
        
        
        # Write all main dishes
        for dish in dish_divs: # for every dish in the list of main dishes
            for link in dish.find_all('a'): #find all the links
                main_url = 'https://www.strongrfastr.com' + link.get('href') #assign url to variable
                driver.get(main_url) #navigate to the URL for the dish on each iteration of the loop
        
                #Wait for the root div to load (the element that contains the rest of the webpage)
                try:
                    element = WebDriverWait(driver, 20).until(
                        EC.presence_of_element_located((By.ID, 'root'))
                    )
                except:
                    driver.quit()
                 
                #Use time to give page a couple seconds to load
                time.sleep(5) # Sleep for 5 seconds
                
                #Save page to variable
                main_dish_source = driver.page_source
        
                #Use BeautifulSoup to parse the webpage
                main_dish_soup = soup(main_dish_source, "html.parser")
                
                
                #Scrape dish names
                dish_name_div = main_dish_soup.find("div", {"class": "max-list ml-center mt0"}) 
                dish_name = dish_name_div.find('h3').text
                
                #Scrape ingredients
                
                main_ingredients_list = main_dish_soup.find_all("div", {"class": "basic-heading"})
                
                #scrape quantities
                
                main_quantities_list = main_dish_soup.find_all("div", {"class": "basic-heading-sub"})[1:]
                # the 0th index returns nutrition info because its tag is a partial match. So we exclude it
                
                main_p = doc.add_paragraph()
                
                main_title = main_p.add_run(dish_name + ': ')
                
                font = main_title.font

                font.name = 'Calibri'
                font.size = Pt(12)
                font.underline = True
                font.all_caps = True

                for i in range(len(main_ingredients_list)):

                    main_p.add_run(main_ingredients_list[i].text + ' ')

                    main_p.add_run(main_quantities_list[i].text + ' ')
                
                for run in main_p.runs[1:]:
                    font = run.font
                    font.name = 'Calibri'
                    font.size = Pt(12)
 

        #Repeat for side dishes
        for side_dish in side_dish_divs: # for every dish in the list of main dishes
            for side_link in side_dish.find_all('a'): #find all the links
                side_url = 'https://www.strongrfastr.com' + side_link.get('href') #assign url to variable
                driver.get(side_url) #navigate to the URL for the dish on each iteration of the loop
        
                #Wait for the root div to load (the element that contains the rest of the webpage)
                try:
                    element = WebDriverWait(driver, 20).until(
                        EC.presence_of_element_located((By.ID, 'root'))
                    )
                except:
                    driver.quit()
        
                #Use time to give page a couple seconds to load
                time.sleep(5) # Sleep for 5 seconds
        
                #Save page to variable
                side_dish_source = driver.page_source
        
                #Use BeautifulSoup to parse the webpage
                side_dish_soup = soup(side_dish_source, "html.parser")
                
                
                #Scrape dish names
                side_name_div = side_dish_soup.find("div", {"class": "max-list ml-center mt0"}) 
                side_name = side_name_div.find('h3').text
                
                #Scrape ingredients
                
                side_ingredients_list = side_dish_soup.find_all("div", {"class": "basic-heading"})
                
                #scrape quantities
                
                side_quantities_list = side_dish_soup.find_all("div", {"class": "basic-heading-sub"})[1:]
                # the 0th index returns nutrition info because its tag is a partial match. So we exclude it
                
                side_p = doc.add_paragraph()
                
                side_title = side_p.add_run(side_name + ': ')
                
                font = side_title.font

                font.name = 'Calibri'
                font.size = Pt(12)
                font.underline = True
                font.all_caps = True
                
                side_ingredients_p = doc.add_paragraph()
                for s in range(len(side_ingredients_list)):
                    
                    side_p.add_run(side_ingredients_list[s].text + ' ')
                    
                    side_p.add_run(side_quantities_list[s].text + ' ')
                    
                    
                for run in side_p.runs[1:]:
                    
                    font = run.font
                    font.name = 'Calibri'
                    font.size = Pt(12)
                
#Close the browser window
driver.close()

#Save our word document
try:
    doc.save('Comidas ' + date_text +'.docx')
except:
    print('File save error')

***Next, to write the code for sending the document via email before finally wrapping it all in a function***

In [32]:
#Istructions on https://www.geeksforgeeks.org/send-mail-attachment-gmail-account-using-python/

fromaddr = 'SomeEmail@example.com'
toaddr = 'SomeEmail@example.com'
   
# instance of MIMEMultipart
msg = MIMEMultipart()
  
# storing the senders email address  
msg['From'] = fromaddr
  
# storing the receivers email address 
msg['To'] = toaddr
  
# storing the subject 
msg['Subject'] = "Comidas"
  
# string to store the body of the mail
body = " "
  
# attach the body with the msg instance
msg.attach(MIMEText(body, 'plain'))
  
# open the file to be sent 
filename = 'Comidas ' + date_text +'.docx'
attachment = open('C:\\Users\\Administrator.Sergio-PC\\Documents\\meal_plans\\'+'Comidas ' + date_text +'.docx', "rb")
  
# instance of MIMEBase and named as p
p = MIMEBase('application', 'octet-stream')
  
# To change the payload into encoded form
p.set_payload((attachment).read())
  
# encode into base64
encoders.encode_base64(p)
   
p.add_header('Content-Disposition', "attachment; filename= %s" % filename)
  
# attach the instance 'p' to instance 'msg'
msg.attach(p)
  
# creates SMTP session
s = smtplib.SMTP('smtp.gmail.com', 587)
  
# start TLS for security
s.starttls()
  
# Authentication
s.login(fromaddr, 'SomePassword')
  
# Converts the Multipart msg into a string
text = msg.as_string()
  
# sending the mail
s.sendmail(fromaddr, toaddr, text)
  
# terminating the session
s.quit()

(221, b'2.0.0 closing connection m205sm416828qke.2 - gsmtp')

***Finally, we wrap it all in a function, add a couple formatting changes, and add status updates along the way***

In [33]:
def get_meals(when = 'next', email = True, send_to = 'SomeEmail@example.com'):
    
    #Library for fetching a webpage
    from urllib.request import urlopen as request

    #Library for parsing html from webpage
    from bs4 import BeautifulSoup as soup 
    
    #Library for writing to a word document
    import docx
    from docx.shared import Pt #for adjusting font and size
    from docx.enum.text import WD_ALIGN_PARAGRAPH #for changing paragraph alignment
    from docx.shared import Inches, Cm #for changing page margains
    
    #Library to pull HTML content from Java enabled sites
    from selenium import webdriver
    
    #Needed to program in waiting for elements to appear before moving on with script
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    
    import re # For regular expressions. Will use to get date of meal plan
    
    import os #for saving docs 
    
    import smtplib #for sending emails
    from email.mime.multipart import MIMEMultipart
    from email.mime.text import MIMEText
    from email.mime.base import MIMEBase
    from email import encoders
    
    import time #add wait time to code
    
    # Status update
    print('STATUS: Dependencies imported.')
    
    # Depending on when parameter, get meal plan url for current of next week
    if when == 'next':
        url = 'https://www.strongrfastr.com/app/meal_plan/next' #save the url we want to open to variable 'url'
    elif when == 'now':
        url = 'https://www.strongrfastr.com/app/meal_plan/' #save the url we want to open to variable 'url'
    
    #Open a browser window
    driver = webdriver.Firefox()
    
    #Navigate to meal planning site
    driver.get(url)
    
    # Status update
    print('STATUS: Webpage open.')
    
    #Wait for the email login button to load
    try:
        element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.ID, 'email-login-btn'))
        )
    except:
        driver.quit()
    
    #Define my email and password
    User_Name = 'SomeEmail@example.com'
    PWD = 'SomePassword'
    
   #Define login button, click button, pass in login credentials
    Login_button = driver.find_element_by_id('email-login-btn')
    Login_button.click()

    username = driver.find_element_by_id("email-field")
    password = driver.find_element_by_id("password-field")

    username.send_keys(User_Name)
    password.send_keys(PWD)
    
    #Wait for sign in button to load
    try:
        element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.ID, 'submit-email-btn'))
        )
    except:
        driver.quit() 
    
    #Define sign in button, click to proceed to my profile and next week's meal plan
    submit_button = driver.find_element_by_id('submit-email-btn')
    submit_button.click()
    
    print('STATUS: Signed in.')
    
    #Wait for the root div to load (the element that contains the rest of the webpage)
    try:
        element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.ID, 'root'))
        )
    except:
        driver.quit()
    
    #Use time to give page a couple seconds to load
    time.sleep(5) # Sleep for 5 seconds
    
    #Save page to variable
    source = driver.page_source
    
    #Use BeautifulSoup to parse the webpage
    page_soup = soup(source, "html.parser")
    
    #Save all meal cards to variable, save the first 4 (One day's meals) to a variable

    #object of all divs that contain a meal
    mydivs = page_soup.find_all("div", {"class": "meal-cards-day-wrap"}) #BeautifulSoup's findall() method returns a list
    
    meals = []

    if len(mydivs) > 0:
        meals = mydivs[0:4] 
        #assign the first four meals (they're the same all week) to variable 'meals'. This is the list we will iterate over
    else:
        print('ERROR:Page not loaded before scraped')
     
    #for each meal, extract the end of it's URL, append to a list
    
    links = []
    for meal in meals:
        for link in meal.find_all('a'):
            links.append(link)
    
    # Get the date for the meal plan in question
    
    link = links[0].get('href')

    dateRegex = re.compile(r'\d\d\d\d-\d\d-\d\d') #defines pattern to search for, creates a regular expression object
    
    date = dateRegex.search(link) #searchses for pattern and returns a match object to a variable
    
    date_text = date.group() # the group() method returns a string we can save to the date_text variable
    
    # Status Update
    print('STATUS: Meals found.')
    
    #Change current working directory

    os.chdir('C:\\Users\\Administrator.Sergio-PC\\Documents\\meal_plans')


    #Create a word document 

    doc = docx.Document()

    # Set to narrow margins
    sections = doc.sections
    for section in sections:
        section.top_margin = Cm(2.54)
        section.bottom_margin = Cm(2.54)
        section.left_margin = Cm(2.54)
        section.right_margin = Cm(2.54)

    date_p = doc.add_paragraph(date_text)

    date_p.alignment = WD_ALIGN_PARAGRAPH.RIGHT

    # Write meal titles and dishes

    # Status list to iterate over
    status = ['25%','50%','75%','100%']
    for meal in meals: #for each meal in the meals list
        title_p = doc.add_paragraph()

        title_p.add_run(meal.div.text + ': ')

        main_dishes = meal.find_all('div',{'class':'meal-plan-recipe-title'}) #create dictionary of all main dishes
        side_dishes = meal.find_all('div',{'class':'low-emphasis lh1 abbr-text'}) #create dictionary of all side dishes
        for dish in main_dishes:

            title_p.add_run(dish.text)

        for side in side_dishes:

            title_p.add_run(side.text)

        for run in title_p.runs:
            font = run.font

            font.name = 'Calibri'
            font.size = Pt(12)
            font.underline = True
            font.all_caps = True


        # Write all ingredients and quantities

        for link in meal.find_all('a'):
            meal_url = 'https://www.strongrfastr.com' + link.get('href')
            driver.get(meal_url) #navigate to the URL for the meal on each iteration of the loop

            #Wait for the root div to load (the element that contains the rest of the webpage)
            try:
                element = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.ID, 'root'))
                )
            except:
                driver.quit()

            #Use time to give page a couple seconds to load
            time.sleep(5) # Sleep for 5 seconds

            #Save page to variable
            meal_source = driver.page_source

            #Use BeautifulSoup to parse the webpage
            meal_page_soup = soup(meal_source, "html.parser")

            #BeautifulSoup's findall() method returns a list
            #object of all divs that contain a dish, and another of side dishes

            dish_divs = meal_page_soup.find_all("div", {"class": "mec-wrapper mb10"}) 
            side_dish_divs = meal_page_soup.find_all("div", {"class": "side-dish-container"})  


            # Write all main dishes
            for dish in dish_divs: # for every dish in the list of main dishes
                for link in dish.find_all('a'): #find all the links
                    main_url = 'https://www.strongrfastr.com' + link.get('href') #assign url to variable
                    driver.get(main_url) #navigate to the URL for the dish on each iteration of the loop

                    #Wait for the root div to load (the element that contains the rest of the webpage)
                    try:
                        element = WebDriverWait(driver, 20).until(
                            EC.presence_of_element_located((By.ID, 'root'))
                        )
                    except:
                        driver.quit()

                    #Use time to give page a couple seconds to load
                    time.sleep(5) # Sleep for 5 seconds

                    #Save page to variable
                    main_dish_source = driver.page_source

                    #Use BeautifulSoup to parse the webpage
                    main_dish_soup = soup(main_dish_source, "html.parser")


                    #Scrape dish names
                    dish_name_div = main_dish_soup.find("div", {"class": "max-list ml-center mt0"}) 
                    dish_name = dish_name_div.find('h3').text

                    #Scrape ingredients

                    main_ingredients_list = main_dish_soup.find_all("div", {"class": "basic-heading"})

                    #scrape quantities

                    main_quantities_list = main_dish_soup.find_all("div", {"class": "basic-heading-sub"})[1:]
                    # the 0th index returns nutrition info because its tag is a partial match. So we exclude it

                    main_p = doc.add_paragraph()

                    main_title = main_p.add_run(dish_name + ': ')

                    font = main_title.font

                    font.name = 'Calibri'
                    font.size = Pt(12)
                    font.underline = True
                    font.all_caps = True

                    for i in range(len(main_ingredients_list)):

                        main_p.add_run(main_ingredients_list[i].text + ' ')

                        main_p.add_run(main_quantities_list[i].text + ' ')

                    for run in main_p.runs[1:]:
                        font = run.font
                        font.name = 'Calibri'
                        font.size = Pt(12)


            #Repeat for side dishes
            for side_dish in side_dish_divs: # for every dish in the list of main dishes
                for side_link in side_dish.find_all('a'): #find all the links
                    side_url = 'https://www.strongrfastr.com' + side_link.get('href') #assign url to variable
                    driver.get(side_url) #navigate to the URL for the dish on each iteration of the loop

                    #Wait for the root div to load (the element that contains the rest of the webpage)
                    try:
                        element = WebDriverWait(driver, 20).until(
                            EC.presence_of_element_located((By.ID, 'root'))
                        )
                    except:
                        driver.quit()

                    #Use time to give page a couple seconds to load
                    time.sleep(5) # Sleep for 5 seconds

                    #Save page to variable
                    side_dish_source = driver.page_source

                    #Use BeautifulSoup to parse the webpage
                    side_dish_soup = soup(side_dish_source, "html.parser")


                    #Scrape dish names
                    side_name_div = side_dish_soup.find("div", {"class": "max-list ml-center mt0"}) 
                    side_name = side_name_div.find('h3').text

                    #Scrape ingredients

                    side_ingredients_list = side_dish_soup.find_all("div", {"class": "basic-heading"})

                    #scrape quantities

                    side_quantities_list = side_dish_soup.find_all("div", {"class": "basic-heading-sub"})[1:]
                    # the 0th index returns nutrition info because its tag is a partial match. So we exclude it

                    side_p = doc.add_paragraph()

                    side_title = side_p.add_run(side_name + ': ')

                    font = side_title.font

                    font.name = 'Calibri'
                    font.size = Pt(12)
                    font.underline = True
                    font.all_caps = True

                    side_ingredients_p = doc.add_paragraph()
                    for s in range(len(side_ingredients_list)):

                        side_p.add_run(side_ingredients_list[s].text + ' ')

                        side_p.add_run(side_quantities_list[s].text + ' ')


                    for run in side_p.runs[1:]:

                        font = run.font
                        font.name = 'Calibri'
                        font.size = Pt(12)
        
        # Status update
        index = meals.index(meal)
        print('STATUS: Meals scraped: ' + status[index])

        # Update index
        #index += 1   
            
    #Close the browser window
    driver.close()

    #Save our word document
    try:
        doc.save('Comidas ' + date_text +'.docx')
    except:
        print('File save error')
    
    # Status update
    print('STATUS: Meal plan document saved.')
    
    # Finally, send our new document to the specified email address
    #Istructions on https://www.geeksforgeeks.org/send-mail-attachment-gmail-account-using-python/
    
    if email == True:
        
        fromaddr = 'SomeEmail@example.com'
        toaddr = send_to

        # instance of MIMEMultipart
        msg = MIMEMultipart()

        # storing the senders email address  
        msg['From'] = fromaddr

        # storing the receivers email address 
        msg['To'] = toaddr

        # storing the subject 
        msg['Subject'] = "Comidas"

        # string to store the body of the mail
        body = " "

        # attach the body with the msg instance
        msg.attach(MIMEText(body, 'plain'))

        # open the file to be sent 
        filename = 'Comidas ' + date_text +'.docx'
        attachment = open('C:\\Users\\Administrator.Sergio-PC\\Documents\\meal_plans\\'+'Comidas ' + date_text +'.docx', "rb")

        # instance of MIMEBase and named as p
        p = MIMEBase('application', 'octet-stream')

        # To change the payload into encoded form
        p.set_payload((attachment).read())

        # encode into base64
        encoders.encode_base64(p)

        p.add_header('Content-Disposition', "attachment; filename= %s" % filename)

        # attach the instance 'p' to instance 'msg'
        msg.attach(p)

        # creates SMTP session
        s = smtplib.SMTP('smtp.gmail.com', 587)

        # start TLS for security
        s.starttls()

        # Authentication
        s.login(fromaddr, 'SomePassword')

        # Converts the Multipart msg into a string
        text = msg.as_string()

        # sending the mail
        s.sendmail(fromaddr, toaddr, text)

        # terminating the session
        s.quit()
        
        # Status update
        print('STATUS: Email sent.')


In [34]:
#Now to test the function
get_meals(when = 'now', email = False)

STATUS: Dependencies imported.
STATUS: Webpage open.
STATUS: Signed in.
STATUS: Meals found.
STATUS: Meals scraped: 25%
STATUS: Meals scraped: 50%
STATUS: Meals scraped: 75%
STATUS: Meals scraped: 100%
STATUS: Meal plan document saved.


***Everything works ! Now we've automated scraping my meal plan, wiritng and saving a word doc, and emailing it! It also works for either current or next week , as well as sending to any email adddress or no email at all.***

***We also added some status updates along the way so we know where we're at in the code's execution at any given point in time***